In [1]:
import xarray as xr
from datetime import date, datetime
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
from pyproj import Transformer, CRS
import time
import glob
import h5py
from pyhdf import SD
from scipy.interpolate import griddata
import calctsi as ct
import pandas as pd
from scipy.stats import circmean, binned_statistic_2d
from IPython import display
from scipy import spatial, stats
from scipy.io import readsav, loadmat
import matplotlib.path as mpath
import cmocean.cm as cmo
import math


In [2]:
#define filepaths

outfp = '/data/users/janheuser/images/'
deffp = '/home/janheuser/projects/def/release_1/'
motvecfp = '/ships19/cryo/janheuser/mot_vec/'
cs2fp = '/ships19/cryo/janheuser/cs2smos/ftp.awi.de/sea_ice/product/cryosat2_smos/v203/nh/'

In [3]:
#define some parameters

proj=ccrs.Stereographic(central_longitude=0, central_latitude=90)

transformer = Transformer.from_crs(4326, 6931)

ease=xr.open_dataset(deffp + 'EASE2_N25km.geolocation.v0.9.nc')
ease['y']=-ease.y
ease = ease.sel(x=slice(-4e6,4e6),y=slice(-4e6,4e6))

y_bins=0.5*ease.y.values[:-1]+0.5*ease.y.values[1:]
y_bins=np.concatenate([[y_bins[0]-np.diff(y_bins)[0]],y_bins,[y_bins[-1]+np.diff(y_bins)[0]]])

x_bins=0.5*ease.x.values[:-1]+0.5*ease.x.values[1:]
x_bins=np.concatenate([[x_bins[0]-np.diff(x_bins)[0]],x_bins,[x_bins[-1]+np.diff(x_bins)[0]]])


In [4]:
#define SLICE functions per Anheuser, et al. (2022) 

def cond_eff(T, S):
    k_a=0.03
    V_a=0.025
    k_i=1.16*(1.91-8.66e-3*T+2.97e-5*T**2)
    k_b=1.16*(0.45+1.08e-2*T+5.04e-5*T**2)
    k_bi=k_i*(2*k_i+k_a-2*V_a*(k_i-k_a))/(2*k_i+k_a+2*V_a*(k_i-k_a))
    T_l = -.0592*S-9.37e-6*S**2-5.33e-7*S**3
    
    return k_bi-(k_bi-k_b)*T_l/T


def stefpred(H_i, T_si, F_w, dur_seconds):
    S = 33
    T_f = -.0592*S-9.37e-6*S**2-5.33e-7*S**3
    L = 333700+762.7*T_f-7.929*T_f**2
    rho = 917
    
    k_eff=cond_eff(T_si-273, 0)
    
    newH = np.sqrt(H_i**2 + (2*k_eff/rho/L)*dur_seconds*(T_f+273-T_si))-dur_seconds*F_w/rho/L
    
    return newH


In [5]:
#Methodology per Section 3

#complete yearly steps
for year in [2010] + list(range(2012,2021)):

    # load in 25 km ice motion
    mot1=xr.open_dataset(motvecfp + 'icemotion_weekly_nh_25km_'+str(year)+'0101_'+str(year)+'1231_v4.1.nc')
    mot2=xr.open_dataset(motvecfp + 'icemotion_weekly_nh_25km_'+str(year+1)+'0101_'+str(year+1)+'1231_v4.1.nc')

    mot = xr.merge([mot1, mot2])
    mot['time'] = mot.indexes['time'].to_datetimeindex()

    #transform motion from EASE 1 to EASE 2
    new_x, new_y = Transformer.from_crs(3408, 6931).transform(mot.x.values, mot.y.values)
    mot['x']=new_x
    mot['y']=new_y

    #replace nans with 0
    mot['u'] = mot['u'].fillna(0)
    mot['v'] = mot['v'].fillna(0)
    
    #define time dimension for the year and create dataset for all data
    if year == 2010:
        time = pd.to_datetime(mot.time.sel(time=slice(str(year)+ "-11-15", str(year+1) + "-4-6")).values)
    else:
        time = pd.to_datetime(mot.time.sel(time=slice(str(year)+ "-10-25", str(year+1) + "-4-6")).values)

    cs2=np.zeros((len(time), len(ease.y),len(ease.x)))
    cs2[:]=np.nan
    cs2_unc=np.zeros((len(time), len(ease.y),len(ease.x)))
    cs2_unc[:]=np.nan
    tsi=np.zeros((len(time), len(ease.y),len(ease.x)))
    tsi[:]=np.nan
    thm=np.zeros((len(time), len(ease.y),len(ease.x)))
    thm[:]=np.nan
    dyn=np.zeros((len(time), len(ease.y),len(ease.x)))
    dyn[:]=np.nan
    dfm=np.zeros((len(time), len(ease.y),len(ease.x)))
    dfm[:]=np.nan
    adv=np.zeros((len(time), len(ease.y),len(ease.x)))
    adv[:]=np.nan
    u=np.zeros((len(time), len(ease.y),len(ease.x)))
    u[:]=np.nan
    v=np.zeros((len(time), len(ease.y),len(ease.x)))
    v[:]=np.nan
    
    thmdyn = xr.Dataset({
        'cs2': xr.DataArray(
                    data   = cs2,
                    dims   = ['time', 'y', 'x'],
                    coords = {'time': time, 'y': ease.y, 'x': ease.x},
                    ),
        'cs2_unc': xr.DataArray(
                    data   = cs2_unc,
                    dims   = ['time', 'y', 'x'],
                    coords = {'time': time, 'y': ease.y, 'x': ease.x},
                    ),
        'tsi': xr.DataArray(
                    data   = tsi,
                    dims   = ['time', 'y', 'x'],
                    coords = {'time': time, 'y': ease.y, 'x': ease.x},
                    ),
        'thm': xr.DataArray(
                    data   = thm,
                    dims   = ['time', 'y', 'x'],
                    coords = {'time': time, 'y': ease.y, 'x': ease.x},
                    ),
        'dyn': xr.DataArray(
                    data   = dyn,
                    dims   = ['time', 'y', 'x'],
                    coords = {'time': time, 'y': ease.y, 'x': ease.x},
                    ),    
        'dfm': xr.DataArray(
                    data   = dfm,
                    dims   = ['time', 'y', 'x'],
                    coords = {'time': time, 'y': ease.y, 'x': ease.x},
                    ),
        'adv': xr.DataArray(
                    data   = adv,
                    dims   = ['time', 'y', 'x'],
                    coords = {'time': time, 'y': ease.y, 'x': ease.x},
                    ),
        'u': xr.DataArray(
                    data   = u,
                    dims   = ['time', 'y', 'x'],
                    coords = {'time': time, 'y': ease.y, 'x': ease.x},
                    ),
        'v': xr.DataArray(
                    data   = v,
                    dims   = ['time', 'y', 'x'],
                    coords = {'time': time, 'y': ease.y, 'x': ease.x},
                    ),
        }
    )

    #load in AWI CS2SMOS data
    for t in time:
        cs2fn=glob.glob(cs2fp + '*EASE2_' + t.strftime('%Y%m%d') + '*.nc')[0]

        cs2=xr.open_dataset(cs2fn)
        cs2['xc']=cs2.xc*1000
        cs2['yc']=cs2.yc*1000

        thmdyn['cs2'].loc[dict(time=t)] = cs2.analysis_sea_ice_thickness.sel(xc=ease.x, yc=ease.y)[0,:,:]
        thmdyn['cs2_unc'].loc[dict(time=t)] = cs2.analysis_sea_ice_thickness_unc.sel(xc=ease.x, yc=ease.y)[0,:,:]
    
    #for each time step within the year
    for i in range(0,len(time)-1):

        print(time[i])
        
        #load in Tsi/SIC
        tsi=[]
        for j in range(0,7):
            tsi.append(ct.calcTsi_amsr(date(time[i].year, time[i].month, time[i].day)+j*pd.Timedelta('1D')).dat)
        tsi=xr.concat(tsi,dim='day').mean(dim='day')
        sic = ct.readamsr_sic(date(time[i].year, time[i].month, time[i].day))
        sic = sic.where(sic.dat!=120)
        sic = sic.where(sic.dat>0)

        #regrid Tsi/ SIC from stereo to EASE
        new_x, new_y = transformer.transform(tsi.lat.values, tsi.lon.values)
        tsi = griddata((new_y.flatten(),new_x.flatten()),tsi.values.flatten(),
                    (ease.y.values[:,None],ease.x.values[None,:]), method='linear')
        tsi = xr.DataArray(tsi, coords=[ease.y, ease.x], dims=['y','x'])

        new_x, new_y = transformer.transform(sic.lat.values, sic.lon.values)
        sic = griddata((new_y.flatten(),new_x.flatten()),sic.dat.values.flatten(),
                    (ease.y.values[:,None],ease.x.values[None,:]), method='linear')
        sic = xr.DataArray(sic, coords=[ease.y, ease.x], dims=['y','x'])

        tsi=tsi.where(sic>95)

        tsi.loc[dict(x=slice(-200000,200000),y=slice(-200000,200000))]=(.5*tsi.loc[dict(
            x=slice(-200000,200000),y=slice(-200000,200000))].interpolate_na(dim='x')+.5*tsi.loc[dict(
            x=slice(-200000,200000),y=slice(-200000,200000))].interpolate_na(dim='y'))
        thmdyn['tsi'][i,:,:] = tsi
                
        #complete SLICE thermodynamic growth estimate and determine other thickness effect estimates
        newH = stefpred(thmdyn['cs2'][i,:,:], tsi, 2, 7*8.64e4)
        newH = newH.where(newH>0).where(sic>95)

        thmdyn['thm'][i,:,:]=newH-thmdyn['cs2'][i,:,:].where(sic>95)
        thmdyn['dyn'][i,:,:]=thmdyn['cs2'][i+1,:,:].where(sic>95)-newH
        thmdyn['adv'][i,:,:]=-(thmdyn['cs2'][i,:,:].differentiate('x')*7*86400e-2*mot.u.sel(time=time[i]).interp_like(newH)+thmdyn['cs2'][i,:,:].differentiate('y')*7*86400e-2*mot.v.sel(time=time[i]).interp_like(newH)).where(sic>95)
        thmdyn['dfm'][i,:,:]=thmdyn['dyn'][i,:,:]-thmdyn['adv'][i,:,:]

        thmdyn['u'][i,:,:]=mot.u.sel(time=time[i]).interp_like(newH)
        thmdyn['v'][i,:,:]=mot.v.sel(time=time[i]).interp_like(newH)


    thmdyn.to_netcdf(deffp + 'thmdyn' + str(year))



/tmp/ipykernel_318897/2662820201.py:11: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'julian', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
  mot['time'] = mot.indexes['time'].to_datetimeindex()


KeyboardInterrupt: 